### An example of LDA

from gensim.test.utils import common_texts

from gensim.corpora.dictionary import Dictionary

> Create a corpus from a list of texts

common_dictionary = Dictionary(common_texts)

common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=10)


In [46]:

import pandas as pd
from gensim.test.utils import common_texts
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
import re
import numpy as np

In [12]:
data = pd.read_csv("data")

In [20]:
import re

# preprocessing function for any song
def preprocessSong(song):
    dataset = ''
    listOfWords = re.split(r'[;,\s...\n()\'!?.]\s*',song) # gets me a list of words
    for word in listOfWords:
#         if word ==  '' || word=='(' || word==')' || word =='\'':
#             pass
#         else:
        word = word.lower()
        if word == 'x2':
            continue
        if word == 'x4':
            continue
        dataset+=' '+word
    return dataset

# function that takes female_songs, male_songs and mixed_songs, returns preprocessed_songs to feed into word2vec
def preprocessAllSongs(song2dMatrix):
    preprocessedSongLyrics = []
    
    for songPair in song2dMatrix:
        artist = songPair[0]
        lyrics = songPair[1]
        preprocessedLyrics = preprocessSong(lyrics)
        
        songsToWords = []
        for word in preprocessedLyrics.split(" "):
            if word!=' ' and word!='':
                songsToWords.append(word)
                
        preprocessedSongLyrics.append(songsToWords)
    return preprocessedSongLyrics


songs = list(data.songLyrics)
artists = list(data.songSinger)
song2dMatrix = []
for x,y in zip(songs, artists):
    song2dMatrix.append([y,x])
preprocessedSongLyrics = preprocessAllSongs(song2dMatrix)
# for songPair in song2dMatrix:
#     print(songPair[1])
    

In [25]:
common_dictionary = Dictionary(preprocessedSongLyrics)

In [37]:
common_corpus = [common_dictionary.doc2bow(text) for text in preprocessedSongLyrics]

# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=5)

In [60]:
some_songs = preprocessedSongLyrics[:10]

# get max probability topic associated with each song in corpus
topic_lda = []
prob = []
for i in range(len(common_corpus)):
    song_lda_input = common_corpus[i]
    topics = lda.get_document_topics(song_lda_input)
    
    maxProb = 0
    maxTopic = 0
    # find the topic with max probability
    # topic[0] is the topic and topic[1] is the probability of that topic
    for topic in topics:
        if(topic[1]>maxProb):
            maxProb = topic[1]
            maxTopic = topic[0]
    topic_lda.append(maxTopic)
    prob.append(maxProb)
    

In [69]:
len(topic_)

,movieName,songName,songSinger,songMusic,songLyricist,songLyrics
0,Bala,Don’t Be Shy,"Badshah, Shalmali Kholgade, Gurdeep Mehendi",Sachin-Jigar,"Mellow D, Badshah","Sun, main hoon thoda sanki\r\nKarun mann ki\r\..."
1,Laal Kaptaan,Lahu Ka Rang Kara,Samira Koppikar,Samira Koppikar,Sahib,Morey.. lahu ka rang kara\r\nMorey lahu ka ran...
2,Laal Kaptaan,Red Red Najariya,Shreya Ghoshal,Samira Koppikar,Saurabh Jain,"Badnaam shehar, badnaam gully\r\nIsme har raat..."
3,Laal Kaptaan,Kaal Kaal,"Brijesh Shandilya, Dino James",Samira Koppikar,Saurabh Jain,"Kaal kaal, kaal kaal, jo sapaat chal raha\r\nW..."
4,Laal Kaptaan,Taandav,"Kailash Kher, Brijesh Shandilya",Samira Koppikar,Puneet Sharma,Shor hai andher mein\r\nJo dher murda pedon ka...
...,...,...,...,...,...,...
3022,Classically Mild,Dhanya Dhanya,Sonu Nigam,Deepak Pandit,Ajay Jhingran,Kaliya khadiya main kaliya ladiya main\r\nKali...
3023,Classically Mild,Lamha Lamha,Sonu Nigam,Deepak Pandit,Ajay Jhingran,Lamha lamha Yeh bole main jeevan hoon\r\nO bho...
3024,Classically Mild,Aye Dil Mat Ro,Sonu Nigam,Deepak Pandit,Ajay Jhingran,"Aye dil mat ro, soch ke tu yeh\r\nHaath mein u..."
3025,Mast 1999,Aasman Kehta Hai Rab Se,Sonu Nigam,Sandeep Chowta,Nitin Raikwar,Aasmaan kehta hai rab se\r\nTune chaand do kyo...


In [71]:
 
data2 = data.copy()
# append this in data
data2['topic_from_lda'] = np.array(topic_lda)
data2['prob_of_topic'] = np.array(prob)

,movieName,songName,songSinger,songMusic,songLyricist,songLyrics,topic_from_lda,prob_of_topic
0,Bala,Don’t Be Shy,"Badshah, Shalmali Kholgade, Gurdeep Mehendi",Sachin-Jigar,"Mellow D, Badshah","Sun, main hoon thoda sanki\r\nKarun mann ki\r\...",2,0.350387
1,Laal Kaptaan,Lahu Ka Rang Kara,Samira Koppikar,Samira Koppikar,Sahib,Morey.. lahu ka rang kara\r\nMorey lahu ka ran...,2,0.869446
2,Laal Kaptaan,Red Red Najariya,Shreya Ghoshal,Samira Koppikar,Saurabh Jain,"Badnaam shehar, badnaam gully\r\nIsme har raat...",1,0.993257
3,Laal Kaptaan,Kaal Kaal,"Brijesh Shandilya, Dino James",Samira Koppikar,Saurabh Jain,"Kaal kaal, kaal kaal, jo sapaat chal raha\r\nW...",3,0.997119
4,Laal Kaptaan,Taandav,"Kailash Kher, Brijesh Shandilya",Samira Koppikar,Puneet Sharma,Shor hai andher mein\r\nJo dher murda pedon ka...,4,0.996224
...,...,...,...,...,...,...,...,...
3022,Classically Mild,Dhanya Dhanya,Sonu Nigam,Deepak Pandit,Ajay Jhingran,Kaliya khadiya main kaliya ladiya main\r\nKali...,1,0.660549
3023,Classically Mild,Lamha Lamha,Sonu Nigam,Deepak Pandit,Ajay Jhingran,Lamha lamha Yeh bole main jeevan hoon\r\nO bho...,3,0.775211
3024,Classically Mild,Aye Dil Mat Ro,Sonu Nigam,Deepak Pandit,Ajay Jhingran,"Aye dil mat ro, soch ke tu yeh\r\nHaath mein u...",3,0.991087
3025,Mast 1999,Aasman Kehta Hai Rab Se,Sonu Nigam,Sandeep Chowta,Nitin Raikwar,Aasmaan kehta hai rab se\r\nTune chaand do kyo...,3,0.996933


In [36]:
other_corpus = [common_dictionary.doc2bow(text) for text in some_songs]
unseen_doc = other_corpus[4]
vector = lda[unseen_doc]
vector

[(4, 0.6007301), (9, 0.39544567)]

In [42]:
for v in vector:
    print(v[0])

4
9


In [34]:
lda.get_document_topics(unseen_doc, per_word_topics=False)

[(3, 0.107754536), (4, 0.31648228), (7, 0.57305765)]

In [40]:
data.shape

(3027, 6)